In [1]:
import pandas as pd
import requests
import time

# Load the transactions data
df = pd.read_excel('../notebooks/cleaned_transactions_processed.xlsx')

# Get unique tickers and currencies
unique_tickers = df['ticker'].unique()

# Base URL for the API endpoints
base_url = 'http://localhost:8000'

# response = requests.get(f"{base_url}/data_ingestion/fetch_and_save_data_exchange_rate/USD/")
# if response.status_code == 200:
#     print(f"Successfully fetched exchange rates for USD")
# else:
#     print(f"Error fetching exchange rates for USD: {response.text}")

# Fetch data for each ticker
for ticker in unique_tickers:
    print(f"Fetching data for {ticker}...")
    
    # Call endpoint to fetch and save daily prices
    response = requests.get(f"{base_url}/data_ingestion/fetch_and_save_data/{ticker}/")
    if response.status_code == 200:
        print(f"Successfully fetched price data for {ticker}")
    else:
        print(f"Error fetching price data for {ticker}: {response.text}")
    
    # Add delay to avoid hitting API rate limits
    time.sleep(1)

print("Completed fetching all required data")



Fetching data for nan...
Error fetching price data for nan: {"status": "error", "message": "'Time Series FX (Daily)'"}
Fetching data for RITM...
Successfully fetched price data for RITM
Fetching data for CRM...
Successfully fetched price data for CRM
Fetching data for NVDA...
Successfully fetched price data for NVDA
Fetching data for SPYD.DE...
Error fetching price data for SPYD.DE: {"status": "error", "message": "'Time Series FX (Daily)'"}
Fetching data for MSFT...
Successfully fetched price data for MSFT
Fetching data for SGLD...
Error fetching price data for SGLD: {"status": "error", "message": "'Time Series (Daily)'"}
Fetching data for WIX...
Error fetching price data for WIX: {"status": "error", "message": "'Time Series (Daily)'"}
Fetching data for AMD...
Error fetching price data for AMD: {"status": "error", "message": "'Time Series (Daily)'"}
Completed fetching all required data


In [13]:
import pandas as pd
import requests
import json
from datetime import datetime

# Path to your Excel file
EXCEL_FILE = "../notebooks/cleaned_transactions_processed.xlsx"  # update if needed
API_URL = "http://localhost:8000/transactions/create/"  # update if deployed
HEADERS = {"Content-Type": "application/json"}


df = pd.read_excel(EXCEL_FILE)

for _, row in df.iterrows():
    if row["ticker"] == "SXLP.MI":
        # Format and clean up data
        payload = {
            "type": row["type"],
            "amount": (row["amount"]),
            "date": row["date"].strftime("%Y-%m-%d") if isinstance(row["date"], (pd.Timestamp, datetime)) else row["date"],
            "user_id": int(row["user_id"]) if not pd.isna(row["user_id"]) else None,
            "ticker": row["ticker"] if pd.notna(row["ticker"]) else None,
            "shares": float(row["shares"]) if pd.notna(row["shares"]) else None,
            "metadata": json.loads(row["metadata"]) if isinstance(row["metadata"], str) and row["metadata"].strip().startswith("{") else {}
        }

        try:
            response = requests.post(API_URL, headers=HEADERS, json=payload)
            if response.status_code == 201:
                print(f"✅ Success: {payload['type']} on {payload['date']} ({payload['amount']} EUR)")
            else:
                print(f"❌ Error {response.status_code} - {response.text} - Data: {payload}")
        except Exception as e:
            print(f"❌ Exception for row: {payload} -> {str(e)}")

✅ Success: sell on 2024-07-12 (36.08 EUR)
✅ Success: sell on 2024-07-12 (505.13 EUR)
✅ Success: buy on 2022-05-30 (-510.39 EUR)
